In [93]:
import os

train_path = '../data/processed/lang_detect/'


In [94]:
with open(os.path.join(train_path, 'collect.words.rus.txt'), 'r') as f:
    rus_words = f.read().split('\n')

with open(os.path.join(train_path, 'collect.words.kbd.txt'), 'r') as f:
    kbd_words = f.read().split('\n')

with open(os.path.join(train_path, 'collect.words.ady.txt'), 'r') as f:
    ady_words = f.read().split('\n')


with open(os.path.join(train_path, 'collect.sentences.rus.txt'), 'r') as f:
    rus_sentences = f.read().split('\n')

with open(os.path.join(train_path, 'collect.sentences.kbd.txt'), 'r') as f:
    kbd_sentences = f.read().split('\n')

with open(os.path.join(train_path, 'collect.sentences.ady.txt'), 'r') as f:
    ady_sentences = f.read().split('\n')

rus_sentences = [sent for sent in rus_sentences if 20 <= len(sent) <= 100]
ady_sentences = [sent for sent in ady_sentences if 20 <= len(sent) <= 100]
kbd_sentences = [sent for sent in kbd_sentences if 20 <= len(sent) <= 100]


In [95]:
import random

for seq in [
    rus_sentences,
    rus_words,
    kbd_sentences,
    kbd_words,
    ady_sentences,
    ady_words
]:
    random.shuffle(rus_sentences)

In [96]:
sent_n = 50000

rus_words_with_label = [f'__label__rus {word}' for word in rus_words]
kbd_words_with_label = [f'__label__kbd {word}' for word in kbd_words]
ady_words_with_label = [f'__label__ady {word}' for word in ady_words]

rus_sentences_with_label = [f'__label__rus {sent}' for sent in rus_sentences[:int(sent_n * 3.0)]]
kbd_sentences_with_label = [f'__label__kbd {sent}' for sent in kbd_sentences[:int(sent_n * 2.5)]]
ady_sentences_with_label = [f'__label__ady {sent}' for sent in ady_sentences[:int(sent_n * 2.0)]]

sentence_with_label = (
        rus_sentences_with_label + rus_words_with_label +
        kbd_sentences_with_label + kbd_words_with_label +
        ady_sentences_with_label + ady_words_with_label
)
random.shuffle(sentence_with_label)

train_sent = sentence_with_label[:int(len(sentence_with_label) * 0.8)]
test_sent = sentence_with_label[int(len(sentence_with_label) * 0.8):]

train_file_path = os.path.join(train_path, 'data.train.txt')
test_file_path = os.path.join(train_path, 'data.test.txt')

with open(train_file_path, 'w') as f:
    f.write('\n'.join(train_sent))

with open(test_file_path, 'w') as f:
    f.write('\n'.join(test_sent))

In [97]:
import fasttext

model = fasttext.train_supervised(input=train_file_path, epoch=100)

Read 2M words
Number of words:  471189
Number of labels: 3
Progress: 100.0% words/sec/thread: 1551353 lr:  0.000000 avg.loss:  0.001156 ETA:   0h 0m 0s


In [98]:
score = model.test_label(test_file_path)
score

{'__label__ady': {'precision': 0.9713499979477076,
  'recall': nan,
  'f1score': 1.9426999958954152},
 '__label__kbd': {'precision': 0.9799635701275046,
  'recall': nan,
  'f1score': 1.9599271402550091},
 '__label__rus': {'precision': 0.8597429693916054,
  'recall': nan,
  'f1score': 1.7194859387832109}}

In [99]:
rus_precision = score['__label__rus']['precision']
kbd_precision = score['__label__kbd']['precision']
ady_precision = score['__label__ady']['precision']

In [100]:
model.save_model(f'../data/processed/lang_detect/models/rus_{rus_precision:.2f}__kbd_{kbd_precision:.2f}__ady_{ady_precision:.2f}.model')

In [101]:
model.quantize()

In [102]:
model.save_model(f'../data/processed/lang_detect/models/rus_{rus_precision:.2f}__kbd_{kbd_precision:.2f}__ady_{ady_precision:.2f}.quant.model')